In [90]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [91]:
original_df = pd.read_csv('2016-2022cabbage_data_file.csv')

# 反轉DataFrame的順序，由上而下改為由下而上
original_df = original_df[::-1].reset_index(drop=True)

高麗菜資料總整理

In [92]:
# 新建一個DataFrame用來存儲新的資料
new_data = []
original_df['finaldate'] = original_df.apply(lambda row: datetime(row['year'], row['month'], row['day']), axis=1)

# 計算106年1月1日至當下日期的天數，並加上專屬號碼
start_date = datetime(2016, 1, 1)  # 106年1月1日的日期
for index, row in original_df.iterrows():
    current_date = row['finaldate']
    days_difference = (current_date - start_date).days + 1
    new_row = {
        'dayoffset': days_difference,
        'TransDate': row['TransDate'],
        'CropCode': row['CropCode'],
        'CropName': row['CropName'],
        'MarketCode': row['MarketCode'],
        'MarketName': row['MarketName'],
        'Upper_Price': row['Upper_Price'],
        'Middle_Price': row['Middle_Price'],
        'Lower_Price': row['Lower_Price'],
        'Avg_Price': row['Avg_Price'],
        'Trans_Quantity': row['Trans_Quantity'],
        'year': row['year'],
        'month': row['month'],
        'day': row['day']
    }
    new_data.append(new_row)

# 將新的資料轉換為DataFrame
new_df = pd.DataFrame(new_data)

# 將結果反轉回原始順序，由下而上改為由上而下
# new_df = new_df[::-1].reset_index(drop=True)

# 將結果寫入新的CSV檔案
new_df.to_csv('new_cabbage.csv', index=False)

合併歷年天氣資料

In [93]:
flag=1
# 創建一個空的 DataFrame 來存放合併後的資料
merged_data = pd.DataFrame()
folder_path = '2017-yilan'
column_names = ["觀測時間(day)", "測站氣壓(hPa)", "海平面氣壓(hPa)", "測站最高氣壓(hPa)", "測站最高氣壓時間(LST)", "測站最低氣壓(hPa)", "測站最低氣壓時間(LST)", "氣溫(℃)", "最高氣溫(℃)", "最高氣溫時間(LST)", "最低氣溫(℃)", "最低氣溫時間(LST)", "露點溫度(℃)", "相對溼度(%)", "最小相對溼度(%)", "最小相對溼度時間(LST)", "風速(m/s)", "風向(360degree)", "最大瞬間風(m/s)", "最大瞬間風風向(360degree)", "最大瞬間風風速時間(LST)", "降水量(mm)", "降水時數(hour)", "最大十分鐘降水量(mm)", "最大十分鐘降水量起始時間(LST)", "最大六十分鐘降水量(mm)", "最大六十分鐘降水量起始時間(LST)", "日照時數(hour)", "日照率(%)", "全天空日射量(MJ/㎡)", "能見度(km)", "A型蒸發量(mm)", "日最高紫外線指數", "日最高紫外線指數時間(LST)", "總雲量(0~10)", "地溫0cm", "地溫5cm", "地溫10cm", "地溫20cm", "地溫30cm", "地溫50cm", "地溫100cm"]

    # 取得資料夾內所有檔案名稱
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, skiprows=0, header=None)
    # df = pd.read_csv(file_path, skiprows=[0])
    df.columns = column_names
    df = df.drop(0)
    df = df.drop(1)
    df = df.drop(2)
    merged_data = pd.concat([merged_data, df])
for i in range (2018,2023):
    # 指定資料夾路徑
    folder_path = '{}-yilan'.format(i)

    # 取得資料夾內所有檔案名稱
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df = df.drop(0)
        merged_data = pd.concat([merged_data, df])


    # 將合併後的資料寫入新的 CSV 檔案
merged_data.to_csv('merged_data.csv', index=False)

print("合併完成！")

合併完成！


添加缺失值標記以及對於2016/1/1的日期差

In [94]:
df = pd.read_csv('merged_data.csv')

df['測站最高氣壓時間(LST)'] = pd.to_datetime(df['測站最高氣壓時間(LST)'], format='%Y/%m/%d %H:%M:%S')

# 計算日期與基準日期（2016/01/01）之間的差異，並新增一個 "日期" 欄位
base_date = pd.to_datetime('2016/01/01', format='%Y/%m/%d')
df['dayoffset'] = (df['測站最高氣壓時間(LST)'] - base_date).dt.days+1
df = pd.concat([df['dayoffset'], df.drop(columns=['dayoffset'])], axis=1)
# 顯示處理後的 DataFrame
# 將處理後的 DataFrame 寫入新的 CSV 檔案
df.to_csv('merged_data.csv', index=False)
print(df['dayoffset'])

0        367
1        368
2        369
3        370
4        371
        ... 
2186    2553
2187    2554
2188    2555
2189    2556
2190    2557
Name: dayoffset, Length: 2191, dtype: int64


平均處理跳過缺失值的

In [95]:
features=["測站氣壓(hPa)","海平面氣壓(hPa)","測站最高氣壓(hPa)","測站最低氣壓(hPa)","氣溫(℃)","最高氣溫(℃)","最低氣溫(℃)","露點溫度(℃)","相對溼度(%)","最小相對溼度(%)","風速(m/s)","風向(360degree)","最大瞬間風(m/s)","最大瞬間風風向(360degree)","降水量(mm)","降水時數(hour)","最大十分鐘降水量(mm)","最大六十分鐘降水量(mm)","日照時數(hour)","全天空日射量(MJ/㎡)","能見度(km)","A型蒸發量(mm)","日最高紫外線指數","總雲量(0~10)","地溫0cm","地溫5cm","地溫10cm","地溫20cm","地溫30cm","地溫50cm","地溫100cm"]
##new_feature=["測站氣壓(hPa)_avg","海平面氣壓(hPa)_avg","測站最高氣壓(hPa)_avg","測站最低氣壓(hPa)_avg","氣溫(℃)_avg","最高氣溫(℃)_avg","最低氣溫(℃)_avg","露點溫度(℃)_avg","相對溼度(%)_avg","最小相對溼度(%)_avg","風速(m/s)_avg","風向(360degree)_avg","最大瞬間風(m/s)_avg","最大瞬間風風向(360degree)_avg","降水量(mm)_avg","降水時數(hour)_avg","最大十分鐘降水量(mm)_avg","最大六十分鐘降水量(mm)_avg","日照時數(hour)_avg","全天空日射量(MJ/㎡)_avg","能見度(km)_avg","A型蒸發量(mm)_avg","日最高紫外線指數_avg","總雲量(0~10)_avg","地溫0cm_avg","地溫5cm_avg","地溫10cm_avg","地溫20cm_avg","地溫30cm_avg","地溫50cm_avg","地溫100cm_avg"]
new_features = [f'{feature}_avg' for feature in features]
pre_features=["Trans_Quantity_day1","Trans_Quantity_day2","Trans_Quantity_day3","Trans_Quantity_day4","Trans_Quantity_day5","Avg_Price_day1","Avg_Price_day2","Avg_Price_day3","Avg_Price_day4","Avg_Price_day5"]
# Avg_Price
# Trans_Quantity

# 迴圈處理每個檔案


# 讀取 CSV 檔案
df = pd.read_csv('merged_data.csv')

dest = pd.read_csv('new_cabbage.csv')
for day in range(548,2558):##2558 733
    target_row =  dest[dest['dayoffset'] == day]
    index=target_row.index
    if target_row.empty:
        continue
    # 初始化每個檔案的溫度總和和有效數據個數
    selected_data = pd.Series([])
    for i in range(5):
        selected_data = pd.concat([selected_data, dest.loc[index - 5 + i, 'Trans_Quantity']])
    for i in range(5):
        selected_data = pd.concat([selected_data, dest.loc[index - 5 + i, 'Avg_Price']])
    dest.loc[dest['dayoffset'] == day, pre_features] = selected_data.values
    dest.loc[dest['dayoffset'] == day, features] = df.loc[df['dayoffset'] == day, features].values
for day in range (0,548):#548
    target_row = dest[dest['dayoffset'] == day]
    if target_row.empty:
        continue
    dest = dest.drop(dest[dest['dayoffset'] == day].index)
##填原本的值回去

##再來是平均後的值
for day in range(548,2558):##2558 733
    target_row = dest[dest['dayoffset'] == day]
    if target_row.empty:
        continue
    # 初始化每個檔案的溫度總和和有效數據個數
    for temp, new_feature in zip(features, new_features):
        temp_sum = 0
        cnt=0
        valid_count = 0
        for num in range (day-60,day):
            goal_row = df[df['dayoffset'] == num]
            if goal_row.empty:
                continue
            # 檢查是否為有效數據
            data=goal_row[temp]
            if not data.isin(["--", "T", "X", "/"]).all():
                temp_sum = temp_sum+float(data)
                valid_count = valid_count + 1
        if valid_count==0:
            avg=0
        else:
            avg=temp_sum/valid_count
        dest.loc[dest['dayoffset'] == day,new_feature] = avg


dest.to_csv('final_training_data.csv', index=False)

<ipython-input-95-60ef68cca221>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  selected_data = pd.Series([])
<ipython-input-95-60ef68cca221>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  selected_data = pd.Series([])
<ipython-input-95-60ef68cca221>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  selected_data = pd.Series([])
<ipython-input-95-60ef68cca221>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  selected_data = pd.Series([])
<ipython-input-95-60ef68cca221>:21: DeprecationWarning: The defa